In [ ]:
import pandas as pd 
train_data = pd.read_csv('../input/sartorius-cell-instance-segmentation/train.csv')
samplesub = pd.read_csv('../input/sartorius-cell-instance-segmentation/sample_submission.csv')
train_data.info()

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(1,1)
train_data.cell_type.value_counts().plot.bar()
ax.set_ylabel('Number of istances')
ax.set_xlabel('Cell types',rotation=0)
fig.tight_layout()
plt.show()

In [ ]:
import tqdm
import os
from termcolor import colored

class config:
    dir = "../input/sartorius-cell-instance-segmentation"
    train_path = dir + '/train'
    test_path = dir +'/test'

def getImagePath(path):
    image_names = []
    for dirn, _, fnames in os.walk(path):
        for fname in fnames:
            fullpath = os.path.join(dirn, fname)
            image_names.append(fullpath)
    return image_names

train_im_path = getImagePath(config.train_path)
test_im_path = getImagePath(config.test_path)

In [ ]:
import cv2
def im_show(im_paths, r, c):
    fig, ax = plt.subplots(nrows = r, ncols = c, 
                          figsize = (16,8))
    for p, im_path in enumerate(im_paths):
        im = cv2.imread(im_path)
        im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
        try: 
            ax.ravel()[p].imshow(im)     # ravel changes a 2-dimensional array or a multi-dimensional array into a contiguous flattened array
            ax.ravel()[p].set_axis_off()
        except:
            continue;
    plt.tight_layout()
    plt.show()

In [ ]:
import numpy as np
def make_mask(mask_files, image_shape=(520, 704), color=False):
    mask = np.zeros(image_shape).ravel()
    for i, mask_file in enumerate(mask_files):
        couples = np.array(mask_file.split()).reshape(-1, 2).astype(int)
        couples[:, 1] = couples[:, 0] + couples[:, 1]
        for couple in couples:
            if color:
                mask[couple[0]: couple[1]] = i
            else:
                mask[couple[0]: couple[1]] = 1
    mask = mask.reshape(520, 704)
    return mask

def plot_image(image_id='0030fd0e6378'):
    fig, ax = plt.subplots(1, 2, figsize=(14,5))
    cell_type = df_train.loc[df_train['id'] == image_id, 'cell_type'][0:1].values
    
    file_name = os.path.join(
        '../input/sartorius-cell-instance-segmentation',
        'train', image_id + '.png')
    image = plt.imread(file_name)
    mask_files = df_train.loc[df_train['id'] == image_id, 'annotation']
    mask = make_mask(mask_files)

    ax[0].imshow(
        image,
        cmap = plt.get_cmap('winter'), 
        origin = 'upper',
        vmax = np.quantile(image, 0.99),
        vmin = np.quantile(image, 0.05)
    )
    ax[0].set_title(f'Source [{image_id}]')
    ax[0].axis('off')
    
    ax[1].imshow(
        image,
        cmap = plt.get_cmap('winter'), 
        origin = 'upper',
        vmax = 255,
        vmin = 0)
    ax[1].imshow(mask, alpha=1, cmap=plt.get_cmap('seismic'))
    ax[1].set_title(f'Source [{image_id}] + Mask {cell_type}')
    ax[1].axis('off')
    plt.show()

df_train = T1

select_image_ids = []
select_image_ids.append(df_train.loc[df_train['cell_type'] == 'astro', 'id'].sample(1).to_list()[0])
select_image_ids.append(df_train.loc[df_train['cell_type'] == 'cort', 'id'].sample(1).to_list()[0])
select_image_ids.append(df_train.loc[df_train['cell_type'] == 'shsy5y', 'id'].sample(1).to_list()[0])

for image_id in select_image_ids:
    plot_image(image_id)

In [ ]:
im_ht, im_width, im_channels = 520, 704, 1

train_ids = train_data['id'].unique().tolist()
test_ids = samplesub['id'].unique().tolist()
# print(test_ids)
# print(train_ids)

In [ ]:
x_train = np.zeros((train_data['id'].nunique(),im_ht, im_width, im_channels), dtype = np.uint8)
y_train = np.zeros((train_data['id'].nunique(),im_ht, im_width, im_channels), dtype = np.uint8)
x_test = np.zeros((samplesub['id'].nunique(),im_ht, im_width, im_channels), dtype = np.uint8)

from tqdm import tqdm

TRAIN_PATH = '../input/sartorius-cell-instance-segmentation/train/'
# https://www.kaggle.com/c/sartorius-cell-instance-segmentation/discussion/291627
def rle_decode(mask_rle, shape=(520, 704, 1)):
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape)  # Needed to align to RLE direction

def rle_encode(img):
    pixels = img.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

for n, id_ in tqdm(enumerate(train_ids), total=len(train_ids)):
    path = TRAIN_PATH + id_
    img = cv2.imread(path + '.png')[:,:]
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY).astype(np.float32) -125
    img = np.expand_dims(img, axis = 2)
    x_train[n] = img
    
    labels = train_data[train_data["id"]
                        == id_]["annotation"].tolist()
    mask = np.zeros((520, 704, 1))
    for label in labels:
        mask += rle_decode(label, shape=(520, 704, 1))
    mask = mask.clip(0, 1)
    y_train[n] = mask

In [ ]:
resize_img = np.zeros((train_data['id'].nunique(),512, 512,im_channels), dtype = np.uint8)
resize_mask = np.zeros((train_data['id'].nunique(),512, 512,im_channels), dtype = np.uint8)
for i in range(len(x_train)):
    img1 = x_train[i]
    mask1 = y_train[i]
    resized_img1 = cv2.resize(img1,(512,512),interpolation = cv2.INTER_AREA)
    resized_img1 = np.expand_dims(resized_img1, axis = 2)
   # print(resized_img1.shape)
    resized_mask1 = cv2.resize(mask1,(512,512),interpolation = cv2.INTER_AREA)
    resized_mask1 = np.expand_dims(resized_mask1, axis = 2)
    #print(resized_mask1.shape)
    resize_img[i] = resized_img1
    resize_mask[i] = resized_mask1
    print(resize_mask.shape)

In [ ]:
x_train = resize_img
y_train = resize_mask

In [ ]:
# Get and resize test images
test_images_id = []
X_test = np.zeros((samplesub['id'].nunique(), im_ht, im_width, im_channels), dtype=np.uint8)
for n, id_ in tqdm(enumerate(test_ids), total=len(test_ids)):
    path = TRAIN_PATH.replace('train', 'test') + id_
    img = cv2.imread(path + '.png')[:,:]
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY).astype(np.float32) -125
    img = np.expand_dims(img, axis = 2)
    print(img.shape)
    X_test[n] = img
    test_images_id.append(id_)

In [ ]:
print(len(X_test))

In [ ]:
resize_test = np.zeros((samplesub['id'].nunique(),512, 512,im_channels), dtype = np.uint8)

for i in range(len(X_test)):
    img1 = X_test[i]
    #print(img1.shape)
    resized_test1 = cv2.resize(img1,(512,512),interpolation = cv2.INTER_AREA)
    resized_test1 = np.expand_dims(resized_test1, axis = 2)
    #print(resized_test1.shape)
    resize_test[i] = resized_test1

X_test = resize_test
print(X_test.shape)

In [ ]:
import numpy as np 
import os
import skimage.io as io
import skimage.transform as trans
import numpy as np
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
# from keras import backend as keras

import keras
# from keras.models import Model,load_model
# # from keras import layers
from tensorflow.keras.losses import BinaryCrossentropy

def unet(pretrained_weights = None,input_size = (512,512,1)):
    inputs = Input(input_size)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    drop5 = Dropout(0.5)(conv5)

    up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
    merge6 = concatenate([drop4,up6], axis = 3)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

    up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
    merge7 = concatenate([conv3,up7], axis = 3)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

    up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    merge8 = concatenate([conv2,up8], axis = 3)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

    up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    merge9 = concatenate([conv1,up9], axis = 3)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)

    model = Model(inputs = inputs, outputs = conv10)

#     model.compile(optimizer = Adam(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])
    model.compile(optimizer = 'adam', loss = BinaryCrossentropy(),metrics = ['accuracy'])
#     model.summary()

    if(pretrained_weights):
    	model.load_weights(pretrained_weights)

    return model

In [ ]:
model = unet()
model.summary()

In [ ]:
from keras.callbacks import EarlyStopping
# Save the best models and its weights at every step 

model_output = os.path.join('./','model.h5')
model_checkpoint = keras.callbacks.ModelCheckpoint(
    model_output,save_best_only = True, save_weights_only = True)

# Reduces the learning rate for no model improvement
lr_reduce = keras.callbacks.ReduceLROnPlateau( monitor='val_loss', factor=0.1, patience=10, verbose=0,
    mode='auto', min_delta=0.0001, cooldown=0, min_lr=0)

es =  EarlyStopping(patience = 10, verbose = 1)

hist = model.fit(x_train, y_train, batch_size = 5,validation_steps=0.5,epochs = 60,callbacks = [EarlyStopping(), model_checkpoint, lr_reduce])

In [ ]:
%checking
print(x_train.shape, y_train.shape)
print(x_train[0:5].shape)
pred = model.predict(x_train[0:5])
#print(pred.shape)
train_preds = (pred > 0.5).astype(np.uint8)
plt.imshow(train_preds[0],cmap = 'gray')

In [ ]:
# Plot model loss
loss = hist.history['loss']
plt.figure()
plt.plot(hist.epoch, loss, 'r', label='Training loss')
plt.title('model loss')
plt.xlabel('epochs')
plt.ylabel('Loss')
plt.legend(['loss'],loc = 'right')
plt.show()

In [ ]:
# testing
preds_test = model.predict(X_test, verbose=1)
preds_test_t = (preds_test >= 0.5).astype(np.uint8)
# Test samples
from random import randint
ix = randint(0, len(preds_test_t)-1)
print(ix)
plt.imshow(X_test[ix])
plt.show()
plt.imshow(np.squeeze(preds_test_t[ix]))
plt.show()